<a href="https://colab.research.google.com/github/mariagrandury/unia-2024/blob/main/creacion_de_datasets_sinteticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets de alineamiento de LLMs en español

## ¿Hay datasets para alinear modelos en español?

Vamos a echar un vistazo a hf.co

## Configuración del entorno

In [ ]:
!pip install huggingface_hub -q
!pip install datasets pandas -q
!pip install "argilla==2.0.0" -q
!pip install trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s 

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

## Creación de datasets sintéticos con Argilla

- Espacio de anotación: https://huggingface.co/spaces/mariagrandury/alineamiento-de-modelos
- Crea tu propio espacio de anotación: http://huggingface.co/new-space?template=argilla/argilla-template-space&name=my-argilla

In [ ]:
import argilla as rg

client = rg.Argilla(
    api_url="https://mariagrandury-alineamiento-de-modelos.hf.space", # URL del Space, termina en *.hf.space
    api_key="<api_key>" # la puedes encontrar en My Settings de tu Argilla Space
)


## Crear un dataset de preguntas y respuestas (SFT)

Formato esperado:
- `prompt`
- `completion`

### 1. Configurar el dataset de Argilla

In [ ]:
import argilla as rg

settings = rg.Settings(
    guidelines="Please, read the prompt carefully and write a response",
    fields = [
        rg.TextField(name="prompt", required=True),
    ],
    questions = [
    rg.TextQuestion(
        name="completion",
        title="Please write an accurate, helpful, and harmless response to the prompt",
        required=True,
    )
    ],
)

dataset = rg.Dataset(
    name="prompts-dataset",
    workspace="argilla", # nombre del workspace creado por defecto
    client=client,
    settings=settings,
)
dataset.create()

Dataset(id=UUID('e06cea79-cb6c-4997-96a2-2df088e0f482') inserted_at=datetime.datetime(2024, 8, 20, 14, 8, 2, 555448) updated_at=datetime.datetime(2024, 8, 20, 14, 8, 4, 61312) name='prompts-dataset' status='ready' guidelines='Please, read the prompt carefully and write a response' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('2f993734-0774-4016-a636-7c11251b2c64') last_activity_at=datetime.datetime(2024, 8, 20, 14, 8, 4, 61312))

### 2. Añadir prompts

- Dataset: https://huggingface.co/datasets/somosnlp/prompt-translation-for-es

In [ ]:
from datasets import load_dataset

prompts = load_dataset('somosnlp/prompt-translation-for-es', split="train[:10]")
prompts.select_columns(["generations"])

def get_generation(example):
    example["generations"] = example["generations"][0]
    return example

new_ds = prompts.map(get_generation)

dataset.records.log(records=new_ds, mapping={"generations": "prompt"}) # mapear columna del dataset con campo del espacio de anotación

Map: 100%|██████████| 10/10 [00:00<00:00, 167.91 examples/s]
/opt/homebrew/lib/python3.11/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['input', 'target', 'target-suggestion', 'target-suggestion-metadata', 'external_id', 'metadata', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")


DatasetRecords: The provided batch size 256 was normalized. Using value 10.

Sending records...: 100%|██████████| 1/1 [00:09<00:00,  9.27s/batch]


DatasetRecords(Dataset(id=UUID('e06cea79-cb6c-4997-96a2-2df088e0f482') inserted_at=datetime.datetime(2024, 8, 20, 14, 8, 2, 555448) updated_at=datetime.datetime(2024, 8, 20, 14, 8, 4, 61312) name='prompts-dataset' status='ready' guidelines='Please, read the prompt carefully and write a response' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('2f993734-0774-4016-a636-7c11251b2c64') last_activity_at=datetime.datetime(2024, 8, 20, 14, 8, 4, 61312)))

## Crear un dataset de preferencia: KTO

Empezamos con el algoritmo sencillito, necesitamos:
- `prompt`: pregunta
- `response`: respuesta
- `label`: respuesta buena o mala

Crear un dataset en Argilla

In [ ]:
import argilla as rg

kto_settings = rg.Settings(
    guidelines="Lee la pregunta y determina si la respuesta está bien o mal teniendo en cuenta...",
    fields = [
        rg.TextField(name="prompt", required=True),
        rg.TextField(name="response", required=True),
    ],
    questions = [
        rg.LabelQuestion(
            name="label",
            title="Valora la respuesta.",
            required=True,
            labels=["👍", "👎"]
        ),
    ],
)

kto_dataset = rg.Dataset(
    name="kto-dataset",
    workspace="argilla", # default name
    client=client,
    settings=kto_settings,
)
kto_dataset.create()

Dataset(id=UUID('5351f515-c6d8-470e-ad49-dda69ef19186') inserted_at=datetime.datetime(2024, 8, 20, 22, 53, 15, 361405) updated_at=datetime.datetime(2024, 8, 20, 22, 53, 16, 461340) name='kpo-dataset-2' status='ready' guidelines='Lee la pregunta y determina si la respuesta está bien o mal teniendo en cuenta...' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('2f993734-0774-4016-a636-7c11251b2c64') last_activity_at=datetime.datetime(2024, 8, 20, 22, 53, 16, 461340))

Push los ejemplos que anotar

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

MODEL = "tiiuae/falcon-7b-instruct"

# Cargar el modelo y el tokenizador
model = AutoModelForCausalLM.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Crear una pipeline de generación de texto
gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Cargar el dataset de prompts
prompts = load_dataset("somosnlp/prompt-translation-for-es", split="train[:10]")

# Formatear según los campos esperados
records = []
for record in prompts:
    prompt = record["generations"][0]
    outputs = gen_pipeline(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    responses = [output["generated_text"] for output in outputs]
    record = {"prompt": prompt, "response": responses[0]}
    records.append(record)

kto_dataset.records.log(records=records)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Crear un dataset de preferencias: DPO

Formato esperado:
- `prompt`
- `response-1`
- `response-2`
- `response_rating`

Crear el dataset en Argilla

<!--
dpo_settings = rg.Settings(
    guidelines="Read the question...",
    fields = [
        rg.TextField(name="prompt", required=True),
        rg.TextField(name="response-1", required=True),
        rg.TextField(name="response-2", required=True),
    ],
    questions = [
        rg.RatingQuestion(
            name="response_ranking",
            title="Rank the responses\n1: first response is better,\n 2: second response is better",
            required=True,
            values=[1, 2]
        ),
    ],
)
-->

In [ ]:
import argilla as rg

dpo_settings = rg.Settings(
    guidelines="Lee la pregunta y...",
    fields = [
        ...
    ],
    questions = [
        ...
    ],
)

dpo_dataset = rg.Dataset(
    name="dpo-dataset",
    workspace="argilla", # default name
    client=client,
    settings=dpo_settings,
)
dpo_dataset.create()

Dataset(id=UUID('1479a93a-91a1-47d4-a059-62e6f9af4b83') inserted_at=datetime.datetime(2024, 8, 20, 16, 19, 6, 71382) updated_at=datetime.datetime(2024, 8, 20, 16, 19, 8, 328168) name='dpo-dataset' status='ready' guidelines='Read the question...' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('2f993734-0774-4016-a636-7c11251b2c64') last_activity_at=datetime.datetime(2024, 8, 20, 16, 19, 8, 328168))

Push los ejemplos que anotar

In [ ]:
...

## Dataset de preferencia más complejo: RLHF

Formato esperado:
- `prompt`
- `response-1`
- `response-2`
- `response_rating`: 1, 2 o 3 en caso de empate
- `correct_response`

Crear el dataset en Argilla

<!--
preference_settings = rg.Settings(
    guidelines="Read the question...",
    fields = [
        rg.TextField(name="prompt", required=True),
        rg.TextField(name="response-1", required=True),
        rg.TextField(name="response-2", required=True)
    ],
    questions = [
        rg.RatingQuestion(
            name="response_ranking",
            title="Rank the responses\n1: first response is better,\n 2: second response is better,\n3: both are equal",
            required=True,
            values=[1, 2,3]
        ),
        rg.TextQuestion(
            name="correct_response",
            title="If none of the responses are helpful and correct, provide the response",
            required=False
        ),
    ],
)
-->


In [ ]:
import argilla as rg

preference_settings = rg.Settings(
    guidelines="Lee la pregunta y...",
    fields = [
        ...
    ],
    questions = [
        ...
    ],
)

preference_dataset = rg.Dataset(
    name="preference-dataset",
    workspace="argilla", # default name
    client=client,
    settings=preference_settings,
)
preference_dataset.create()


Push los ejemplos que anotar

In [ ]:
...

Push el dataset al Hub de Hugging Face

In [ ]:
preference_dataset = client.datasets(name="preferences-dataset")
preference_dataset.to_hub(repo_id="<hf_handle>/<nombre_del_dataset>")